There are a number of pre-computed databases here: https://benlangmead.github.io/aws-indexes/k2 . For demonstration purposes, let us use one of the smallest ones : Standard-8. This has already been downloaded in data/kraken2_db/

In [4]:
! mkdir -p results/kraken2_output

To run on one sample, you would do :

`kraken2 --paired --db data/kraken2_db/standard-8/ data/metagenome_samples/FH1_1.fastq.gz data/metagenome_samples/FH1_2.fastq.gz --output results/kraken2_output/FH1`

To run on all at once, use the script below.

In [9]:
!for sample in $(ls data/metagenome_samples/*_1.fastq.gz | sed 's/.*\///' | sed 's/_1.*//'); do \
    echo "Processing sample: ${sample}"; \
    kraken2 --paired --db data/kraken2_db/standard-8/  --output results/kraken2_output/${sample}.kraken.output --report results/kraken2_output/${sample}.kraken.report \
    data/metagenome_samples/${sample}_1.fastq.gz data/metagenome_samples/${sample}_2.fastq.gz;\
done

Processing sample: BH1
Loading database information... done.
1000000 sequences (302.00 Mbp) processed in 11.296s (5311.5 Kseq/m, 1604.07 Mbp/m).
  705810 sequences classified (70.58%)
  294190 sequences unclassified (29.42%)
Processing sample: BH2
Loading database information... done.
1000000 sequences (302.00 Mbp) processed in 11.099s (5406.0 Kseq/m, 1632.60 Mbp/m).
  336398 sequences classified (33.64%)
  663602 sequences unclassified (66.36%)
Processing sample: BH3
Loading database information... done.
1000000 sequences (302.00 Mbp) processed in 10.508s (5709.9 Kseq/m, 1724.40 Mbp/m).
  208838 sequences classified (20.88%)
  791162 sequences unclassified (79.12%)
Processing sample: BH4
Loading database information... done.
1000000 sequences (302.00 Mbp) processed in 10.736s (5588.9 Kseq/m, 1687.84 Mbp/m).
  261480 sequences classified (26.15%)
  738520 sequences unclassified (73.85%)
Processing sample: FH1
Loading database information... done.
1000000 sequences (302.00 Mbp) processe

In [10]:
!head  results/kraken2_output/BH1.report 


 29.42	294190	294190	U	0	unclassified
 70.58	705810	485	R	1	root
 70.50	704961	2859	R1	131567	  cellular organisms
 68.09	680862	29099	D	2	    Bacteria
 63.41	634069	38696	P	1224	      Pseudomonadota
 55.35	553477	2461	C	28216	        Betaproteobacteria
 54.62	546158	9330	O	80840	          Burkholderiales
 51.18	511791	12223	F	506	            Alcaligenaceae
 49.69	496927	34939	G	222	              Achromobacter
 44.69	446934	1893	G1	2626865	                unclassified Achromobacter


bracken -d data/kraken2_db/standard-8/ -l G -i results/kraken2_output/BH1.report -o results/kraken2_output/BH1.bracken.output

In [ ]:
!for sample in $(ls data/metagenome_samples/*_1.fastq.gz | sed 's/.*\///' | sed 's/_1.*//'); do \
    bracken -d /data/kraken2-db/standard-8 -l G -i results/kraken2_output/${sample}.kraken.report -r 100 -o results/kraken2_output/${sample}.bracken.output;\
done